## Problem: Random samples for a finite  population of bitstrings

Given a number of bits, write the `get_sample` function to return a list `n` of random samples from a finite probability mass function defined by a dictionary with keys defined by a specified number of bits. For example, given `3` bits, we have the following dictionary that defines the probability of each of the keys,

    ['000', '001', '010', '011', '100', '101', '110', '111']

The values of the dictionary correspond of the probability of drawing any one of these. For example, if all of these were equally likely, then here is the corresponding dictionary `p`,

    p={'000': 0.125,
     '001': 0.125,
     '010': 0.125,
     '011': 0.125,
     '100': 0.125,
     '101': 0.125,
     '110': 0.125,
     '111': 0.125}
 
 
 Your `get_sample` function should return something like the following,
 
     >>> get_sample(nbits=3,prob=p,n=4)
     ['101', '000', '001', '100']
 
**Hint**: Validate your inputs thoroughly. 

In [1]:
import numpy as np
import itertools
import math

In [2]:
def get_sample(nbits=3,prob=None,n=1):
    '''
    Given a number of bits, generate a n-sampled list with the given probability.
    
    :param: nbits 
    :type : int
    :param: prob
    :type:  dict
    :param: n
    :type:  int    
    '''
    assert isinstance(nbits,int)
    assert isinstance(prob,dict)
    assert isinstance(n,int)
    assert nbits>0
    assert n>0
    
    bitstrings = ["".join(seq) for seq in itertools.product("01", repeat=nbits)]
    #print(bitstrings)
    
    key = list(prob.keys())
    #print(key)
    
    assert len(key) == len(bitstrings)
    assert set(key) == set(bitstrings)
    
    value = list(prob.values())
    #print(sum(value))
    
    assert sum(value) == 1
    
    sample = np.random.choice(key,n,p=value)
    return list(sample)
    
    
p={'000': 0.125, '001': 0.125, '010': 0.125, '011': 0.125, '100': 0.125, '101': 0.125, '110': 0.125, '111': 0.125}
get_sample(nbits=3,prob=p,n=4) 

['100', '010', '111', '010']

## Map bitstrings 

Write a function `map_bitstring` that takes a list of bitstrings (i.e., `0101`) and maps each bitstring to `0` if the number of `0`s in the bitstring strictly exceeds the number of `1`s. Otherwise, map that bitstring to `1`. The output of your function is a dictionary of the so-described key-value pairs.

Here is an example:

    >>> x= ['100', '100', '110', '010', '111', '000', '110', '010', '011', '000']
    >>> map_bitstring(x)
    {'100': 0, '110': 1, '010': 0, '111': 1, '000': 0, '011': 1}


In [3]:
def map_bitstring(instrings):
    '''
    Given a list of bitstrings, generate the corresponding map bitstrings.
    
    :param: instrings 
    :type : list
    '''
    assert isinstance(instrings,list)
    assert all(isinstance(x,str) for x in instrings)
    
    nbits = len(instrings[0])
    bitstrings = ["".join(seq) for seq in itertools.product("01", repeat=nbits)]
    
    assert set(instrings).issubset(set(bitstrings))
    
    instr = list(set(instrings))
    
    map_dict = {}
    for s in instr:
        if s.count('0')>s.count('1'):
            map_dict[s] = 0
        else:
            map_dict[s] = 1
    
    return map_dict


x= ['100', '100', '110', '010', '111', '000', '110', '010', '011', '000']
map_bitstring(x)

{'111': 1, '000': 0, '100': 0, '010': 0, '110': 1, '011': 1}

## Gather by values 

Now that you have `get_sample` working, generate `n` samples and tally the number of times an existing key is repeated. Generate a new dictionary with bitstrings as keys and with values as lists that contain the corresponding mapped values from `map_bitstring`. Here is an example for  `n=20`,

    >>> x=get_sample(nbits=2,prob={'00':1/4,'01':1/4,'10':1/4,'11':1/4},n=20)
    >>> print(x)
    ['10', '11', '01', '00', '10', '00', '00', '11', '10', '00', '00', '01', '01', '11', '10', '00', '11', '10', '11', '11']

Write a function `gather_values` that can produce the following output from `x`:

    {'10': [1, 1, 1, 1, 1],
     '11': [1, 1, 1, 1, 1, 1],
     '01': [1, 1, 1],
     '00': [0, 0, 0, 0, 0, 0]}

In [4]:
def gather_values(seq):
    '''
    Given a list of bitstrings, generate a dictionary with bitstrings as keys and with values as lists 
    that contain the corresponding mapped values.
    
    :param: seq 
    :type : list
    '''
    assert isinstance(seq,list)
    assert all(isinstance(x,str) for x in seq)
    
    nbits = len(seq[0])
    bitstrings = ["".join(s) for s in itertools.product("01", repeat=nbits)]
    
    assert set(seq).issubset(set(bitstrings))
    
    map_dict = map_bitstring(seq)
    
    map_tally = {}
    l = list(set(seq))
    for s in l:
        map_tally[s] = []
        
    for s in seq:
        map_tally[s].append(map_dict.get(s))
        
    return map_tally
    
    
x = ['10', '11', '01', '00', '10', '00', '00', '11', '10', '00', '00', '01', '01', '11', '10', '00', '11', '10', '11', '11']
gather_values(x)

{'00': [0, 0, 0, 0, 0, 0],
 '11': [1, 1, 1, 1, 1, 1],
 '01': [1, 1, 1],
 '10': [1, 1, 1, 1, 1]}

## Threshold sample counts

From `gather_values`, we can group the results of the random samples. Now, we want to threshold those values based upon their frequency and value. Given `threshold=2`, we want to keep only those bitstrings with the two highest frequency counts of the `1` value. For example, as before we have,

    x=['10', '11', '01', '00', '10', '00', '00', '11', '10', '00', '00', '01', '01', '11', '10', '00', '11', '10', '11', '11']
    
according to our last result, we had 

    {'10': [1, 1, 1, 1, 1],
     '11': [1, 1, 1, 1, 1, 1],
     '01': [1, 1, 1],
     '00': [0, 0, 0, 0, 0, 0]}

But because the `threshold=2`, we only want to keep the bitstrings with the two most frequent `1`s and set all of the rest to `0`. In this case, this is `10` and `11` with the following output:

    {'10': 1,
     '11': 1,
     '01': 0,
     '00': 0}
    
Note that `01` corresponding value was set to `0` because it did not have the top two most frequent values of `1`. If there is a tie, then use the smallest value the tied bitstrings to pick the winner. Here is a more detailed example:

    x= ['1111', '0110', '1001', '0011', '0111', '0100', '0111', '1100', '0011', '0010', '0010', '1010', '1010', '1100', '0110', '0101', '0110', '1111', '1001', '0110', '0010', '1101', '0101', '0010', '0100', '0010', '0000', '0000', '0011', '0110', '0101', '1010', '1011', '1101', '1100', '0111', '1110', '0100', '0110', '1101', '0001', '1110', '0010', '0001', '1010', '1010', '0011', '1000', '0010', '0000', '1010', '1101', '1111', '1000', '1000', '0010', '1010', '0101', '0101', '1101', '0110', '1001', '1100', '1100', '1000', '1010', '0011', '0101', '0101', '0011', '0001', '1010', '0011', '0011', '1101', '1010', '0101', '0011', '1011', '0101', '0000', '1111', '1001', '0101', '1100', '0011', '1111', '1101', '0001', '1111', '1110', '1111', '0001', '0010', '0110', '0100', '0101', '1100', '1110', '1001']

With corresponding output from `threshold_values`,

    >>> threshold_values(seq,3)
    {'0000': 0, '0001': 0, '0010': 0, '0011': 1, '0100': 0, '0101': 1, '0110': 0, '0111': 0, '1000': 0, '1001': 0, '1010': 1, '1011': 0, '1100': 0, '1101': 0, '1110': 0, '1111': 0}


In [5]:
def threshold_values(seq,threshold=1):
    '''
    Given a list of bitstrings, generate a gathered_value dictionary of those bitstrings 
    with the two highest frequency counts of the 1 value.
    
    :param: seq 
    :type : list
    :param: threshold 
    :type : int
    '''
    assert isinstance(seq,list)
    assert all(isinstance(x,str) for x in seq)
    assert isinstance(threshold,int)
    
    nbits = len(seq[0])
    bitstrings = ["".join(s) for s in itertools.product("01", repeat=nbits)]
    
    assert set(seq).issubset(set(bitstrings))
    assert threshold>0 and threshold<=math.pow(2,nbits)
    
    map_tally = gather_values(seq)
    #print(map_tally)
    
    for k in map_tally.keys():
        map_tally[k] = map_tally.get(k).count(1)
    #print(map_tally)
    
    map_dict = dict(sorted(map_tally.items(), key=lambda x:x[1], reverse=True))
    #print(map_dict)
    
    for k in map_dict.keys():
        threshold -= 1
        if map_dict[k]!=0:
            if threshold>=0:
                map_dict[k] = 1
            else:
                map_dict[k] = 0
    
    return map_dict
    
    
#x=['10', '11', '01', '10', '10', '00', '00', '11', '10', '00', '00', '01', '01', '11', '10', '00', '11', '10', '11', '11']
#threshold_values(x,1)

x= ['1111', '0110', '1001', '0011', '0111', '0100', '0111', '1100', '0011', '0010', '0010', '1010', '1010', '1100', '0110', '0101', '0110', '1111', '1001', '0110', '0010', '1101', '0101', '0010', '0100', '0010', '0000', '0000', '0011', '0110', '0101', '1010', '1011', '1101', '1100', '0111', '1110', '0100', '0110', '1101', '0001', '1110', '0010', '0001', '1010', '1010', '0011', '1000', '0010', '0000', '1010', '1101', '1111', '1000', '1000', '0010', '1010', '0101', '0101', '1101', '0110', '1001', '1100', '1100', '1000', '1010', '0011', '0101', '0101', '0011', '0001', '1010', '0011', '0011', '1101', '1010', '0101', '0011', '1011', '0101', '0000', '1111', '1001', '0101', '1100', '0011', '1111', '1101', '0001', '1111', '1110', '1111', '0001', '0010', '0110', '0100', '0101', '1100', '1110', '1001']
threshold_values(x,3)
#ds=dict(sorted(d.items(),key=lambda x:x[0]))

#x=['0','0','0','0','0','0','0']
#threshold_values(x,2)

{'0101': 1,
 '0011': 1,
 '1010': 1,
 '0110': 0,
 '1101': 0,
 '1111': 0,
 '1100': 0,
 '1001': 0,
 '1110': 0,
 '0111': 0,
 '1011': 0,
 '1000': 0,
 '0000': 0,
 '0001': 0,
 '0010': 0,
 '0100': 0}